In [1]:
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx

In [ ]:
os.chdir('../data/TransportationNetworks')
root = os.getcwd()
root

In [ ]:
os.getcwd()

In [27]:
def create_constraints(net_name):
    os.chdir(f'./{net_name}')
    df = pd.read_csv(f'{net_name}_net.csv', comment='#')

    constraints = df[['init_node', 'term_node']].copy(deep=True)
    constraints['lower_bound'] = df['capacity'] * 0.9
    constraints['upper_bound'] = df['capacity'] * 1.1
    constraints['investment_cost_param'] = 0
    constraints.to_csv(f'{net_name}_constraints.csv', index=False)

    os.chdir(f'..')
    return constraints

In [ ]:
constraints = create_constraints('SiouxFalls')
constraints

In [3]:
def calculate_lambda_for_middle_rows(df):
    """Вычисляет λ для строк с bound_type = 'Middle'"""
    middle_mask = df['bound_type'] == 'Middle'
    
    if not middle_mask.any():
        raise ValueError("Нет строк с bound_type = 'Middle'. Нельзя вычислить среднее λ.")
    
    # Вычисляем λ по формуле: λ = (condition_result * 38000 / 5) - 1
    df['lambda'] = np.nan
    df.loc[middle_mask, 'lambda'] = (df.loc[middle_mask, 'condition_result'] * 38000 / 5) - 1
    
    return df

def check_condition(row):
    """
    Проверяет соответствие bound_type и comparison_result
    """
    bound = row['bound_type']
    comparison = row['comparison_result']
    
    if bound == 'Lower' and comparison == 'Less':
        return True
    elif bound == 'Middle' and comparison == 'Equal':
        return True
    elif bound == 'Upper' and comparison == 'More':
        return True
    else:
        return False

def get_optimality(net_name):
    df = pd.read_csv(f'{net_name}/{net_name}_optimality.csv', comment='#')
    threshold = 0.01

    upper_condition = abs(df['capacity'] - df['upper_bound']) <= threshold
    lower_condition = abs(df['capacity'] - df['lower_bound']) <= threshold

    conditions = [
        upper_condition,
        lower_condition
    ]

    choices = [
        'Upper',
        'Lower'
    ]

    df['bound_type'] = np.select(conditions, choices, default='Middle')

    cols = df.columns.tolist()

    idx1 = cols.index('condition_result')
    idx2 = cols.index('bound_type')

    cols[idx1], cols[idx2] = cols[idx2], cols[idx1]

    df = df[cols]

    threshold=0.000001
    # Вычисляем λ для строк Middle
    df = calculate_lambda_for_middle_rows(df)
    
    # Находим среднее λ из строк Middle
    middle_lambdas = df.loc[df['bound_type'] == 'Middle', 'lambda']
    if len(middle_lambdas) == 0:
        raise ValueError("Нет значений λ для строк Middle")
    
    mean_lambda = middle_lambdas.mean()
    print(f"Среднее λ из всех строк с 'Middle': {mean_lambda:.6f}")
    
    # Вычисляем эталонное значение
    reference_value = (1 + mean_lambda) * 5 / 38000
    print(f"Эталонное значение (reference): {reference_value:.8f}")
    print(f"Порог сравнения (threshold): ±{threshold}")
    
    # Создаем временную колонку с разницей
    df['diff'] = df['condition_result'] - reference_value
    
    # Определяем comparison_result на основе порога
    conditions = [
        df['diff'].abs() <= threshold,  # равно (в пределах порога)
        df['diff'] > threshold,         # больше
        df['diff'] < -threshold         # меньше
    ]
    
    choices = ['Equal', 'More', 'Less']
    
    df['comparison_result'] = np.select(conditions, choices, default='Equal')
    
    # Удаляем временную колонку diff (если не нужна)
    df = df.drop('diff', axis=1)

    df['condition_check'] = df.apply(check_condition, axis=1)

    df.to_excel(f'{net_name}/{net_name}_optimality.xlsx', index=False) 

    return df

In [ ]:
df = get_optimality('SiouxFalls_SingleODPair_0')

In [ ]:
df 

In [51]:
middle_rows = df[df['bound_type'] == 'Middle']
first_middle = middle_rows.iloc[0]
condition_result = first_middle['condition_result']

# Вычислить λ
lambda_value = (condition_result * 38000 / 5) - 1

In [ ]:
lambda_value

In [ ]:
middle_rows = df[df['bound_type'] == 'Middle']
print("\nВсе строки с 'Middle':")
for idx, row in middle_rows.iterrows():
    lambda_i = (row['condition_result'] * 38000 / 5) - 1
    print(f"Строка {idx}: condition_result={row['condition_result']:.6f}, λ={lambda_i:.6f}")